In [1]:
import pandas as pd
import numpy as np

# Import etl.py module

In [2]:
# add src to module search path 

import os
import sys

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

if os.name == "nt":
    path_separator = "\\"
else:
    path_separator = "/"
    
module_path = f"{parent_dir}{path_separator}src"
sys.path.append(module_path)

# import custom module
from etl import DataPreprocessing

# How to use the module

To demonstrate the features of the module, we perform a simplified ETL pipeline, where we extract raw data from a csv, remove duplicate rows, and store the tranformed dataset in a csv file.

In [3]:
file_path = f'..{path_separator}data{path_separator}raw{path_separator}interview_signup.csv'

# class instantiation
Pipeline_1 = DataPreprocessing(file_path)

In [4]:
# load raw data from csv
sep = ','
header = 0
dtype = {'original_product_name': str,
         'postcode'             : str,
         'bundesland'           : str,
         'total_bonus'          : 'float64',
         'order_date'           : str}

Pipeline_1.load_data_from_csv(# encoding='utf-8', 
                            sep=sep,
                            header=header,
                            dtype=dtype)
Pipeline_1.df.shape

(318345, 5)

In [5]:
# column names
Pipeline_1.df.columns

Index(['original_product_name', 'postcode', 'bundesland', 'total_bonus',
       'order_date'],
      dtype='object')

Check data types

In [6]:
Pipeline_1.df.dtypes

original_product_name     object
postcode                  object
bundesland                object
total_bonus              float64
order_date                object
dtype: object

In [7]:
Pipeline_1.df['original_product_name'].unique()

array(['E.ON STROM', 'E.ON STROM ÖKO', 'E.ON STROM ÖKO 24',
       'E.ON STROM 24', 'E.ON STROM PUR', 'E.ON STROM Ã–KO',
       'E.ON STROM 24 24 24', 'E.ON STROM 24 24', 'E.ON STROM ÖO',
       'E.ON STROM 24 24 24 24 24 24 24'], dtype=object)

In [8]:
Pipeline_1.df['bundesland'].unique()

array(['Nordrhein-Westfalen', 'Baden-Württemberg', 'Hessen', 'Berlin',
       'Schleswig-Holstein', 'Niedersachsen', nan, 'Bayern',
       'Rheinland-Pfalz', 'Sachsen', 'Bremen', 'Brandenburg', 'Thüringen',
       'Saarland', 'Mecklenburg-Vorpommern', 'Hamburg', 'Sachsen-Anhalt'],
      dtype=object)

In [13]:
Pipeline_1.df['order_date'].apply(lambda x: type(x)).unique()


array([<class 'datetime.date'>], dtype=object)

In [11]:
# order_date contains all strings
Pipeline_1.df['order_date'].apply(lambda x: type(x)).unique()

# convert order_date column to datetime.date
Pipeline_1.df['order_date'] = pd.to_datetime(Pipeline_1.df['order_date'], format="%Y-%m-%d")
Pipeline_1.df['order_date'] = Pipeline_1.df['order_date'].dt.date

In [25]:
# check conversion status
Pipeline_1.df.info()  # alt.: Pipeline_1.df.dtypes

<class 'pandas.core.frame.DataFrame'>
Index: 318174 entries, 0 to 318344
Data columns (total 5 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   original_product_name  318174 non-null  category
 1   postcode               318174 non-null  object  
 2   bundesland             288654 non-null  category
 3   total_bonus            318174 non-null  float64 
 4   order_date             318174 non-null  object  
dtypes: category(2), float64(1), object(2)
memory usage: 10.3+ MB


Remove duplicate rows:

In [15]:
# delete duplicate rows
Pipeline_1.remove_duplicate_rows()
Pipeline_1.df.shape

(318175, 5)

Check missing data

In [16]:
# check number of missing values for each column
columns = list(Pipeline_1.df.columns)
Pipeline_1.missing_values(columns).sum()

# alt.: my_class.df.info()

original_product_name        0
postcode                     0
bundesland               29521
total_bonus                  0
order_date                   0
dtype: int64

In [17]:
# returns logical index of all rows with missing state
idx_missing_state = Pipeline_1.missing_values('bundesland')

# Return sample of filled states
Pipeline_1.df.loc[~idx_missing_state, 'bundesland'].sample(10)

103945                    Bayern
72210              Niedersachsen
149490                    Bremen
9957           Baden-Württemberg
178793                    Bayern
152559           Rheinland-Pfalz
302554    Mecklenburg-Vorpommern
56796          Baden-Württemberg
303500           Rheinland-Pfalz
6383                   Thüringen
Name: bundesland, dtype: category
Categories (16, object): ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', ..., 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']

In [19]:
# count number of valid German states 
Pipeline_1.validate_state('bundesland').sum()

288813

Inspect the invalid postcode cases:

In [18]:
idx_valid_postcodes = Pipeline_1.validate_postcode("postcode", r"^[0-9]{5}$")
print("Number of valid postcodes: ", idx_valid_postcodes.sum())

# return sample of 20 invalid postcodes
Pipeline_1.df.loc[~idx_valid_postcodes, 'postcode'].sample(20)

Number of valid postcodes:  226865


295630    31275.0
79595     22885.0
24809        4564
162159    94527.0
107074    73550.0
215492    44625.0
45561     57072.0
174003    94491.0
164772    18437.0
212834    73262.0
147558    13437.0
206993    91207.0
303832    75181.0
87033     17039.0
144120    32756.0
107416       4736
287849    37412.0
80465     26842.0
132544    98553.0
253176    89155.0
Name: postcode, dtype: object

In [19]:
# Remove decimals and check again
Pipeline_1.remove_decimals('postcode')

# remaining invalid postcodes
idx_valid_postcodes = Pipeline_1.validate_postcode('postcode')
print("Remaining invalid postcodes: ", Pipeline_1.df.shape[0] - idx_valid_postcodes.sum())

# show sample of 20 remaining invalid cases
Pipeline_1.df.loc[~idx_valid_postcodes, 'postcode'].sample(20)

78598 entries were changed.

Remaining invalid postcodes:  16610


30350     6536
91588     8525
109080    4177
129002    8485
161377    1796
7203      4860
202595    4509
176469    4600
33509     8248
46030     4509
42752     6484
63876     4107
184571    6484
106389    9243
316635    8491
184465    9244
177557    6842
277310    4808
9371      4519
62982     1968
Name: postcode, dtype: object

In [20]:
# check for postcodes with more than 5 digits and less than 4 digits
idx_postcode_unequal_5 = (Pipeline_1.df['postcode'].str.len()>5) | (Pipeline_1.df['postcode'].str.len()<4)

Pipeline_1.df.loc[idx_postcode_unequal_5, 'postcode']

266922    92696JAVAS
Name: postcode, dtype: object

In [21]:
# drop record with index 266922
Pipeline_1.df.drop(index=266922, inplace=True)

# remaining invalid postcodes
idx_valid_postcodes = Pipeline_1.validate_postcode('postcode')
print("Remaining invalid postcodes: ", Pipeline_1.df.shape[0] - idx_valid_postcodes.sum())

Remaining invalid postcodes:  16609


In [22]:
# pad postcodes with zero from left
Pipeline_1.zero_padding('postcode', side='left', n=5)

16609 entries were changed.



In [23]:
# remaining invalid postcodes
idx_valid_postcodes = Pipeline_1.validate_postcode('postcode')
print("Remaining invalid postcodes: ", Pipeline_1.df.shape[0] - idx_valid_postcodes.sum())

# have a look at cleaned column
Pipeline_1.df[idx_valid_postcodes].sample(20)

Remaining invalid postcodes:  0


In [20]:
# store preprocessed csv in folder data/processed
file_path_processed = f"..{path_separator}data{path_separator}processed{path_separator}demo_etl_module_processed.csv"
Pipeline_1.save_data_to_csv(file_path_processed, index=False)